In [1]:
import numpy as np
import pandas as pd
import torch
import json
import glob
import os

from tqdm import tqdm
from transformers import BertModel, BertTokenizer

import re
from functools import partial
from multiprocessing import Pool, cpu_count

import nltk
import statsmodels.api as sm
from nltk.stem.snowball import SnowballStemmer

from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer
from transformers import BertModel, BertTokenizer

## Functions

In [2]:
def multiprocess_tasks(func, tasks, data, njobs=cpu_count()):
    wrapper = partial(func, df=data)
    res = []
    with Pool(processes=njobs) as p:
        n = len(tasks)
        with tqdm(total=n) as pbar:
            for i, v in enumerate(p.imap_unordered(wrapper, tasks)):
                res.append(v)
                pbar.update()
    return res

def divide_chunks(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n]

def r_squared_adj(data, group, dummy=True, var_name="palt"):
    y = data[var_name]
    if dummy:
        x = data[group]
    else:
        x = pd.get_dummies(data[group], drop_first=True)
    x = sm.add_constant(x)
    mod = sm.OLS(y, x).fit()
    return mod.rsquared_adj.round(2)


def match_token(text, token):
    tokens = tokenize_only(text)
    if token in tokens:
        return 1
    else:
        return 0


def token_power(token, df, desc_column="award_description"):
    df[token] = df[desc_column].apply(match_token, token=token)
    r2 = r_squared_adj(df, token)
    return {token: r2}


def tokenize_and_stem(text, stopwords=None):
    stemmer = SnowballStemmer("english")
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [
        word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)
    ]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search("[a-zA-Z]", token):
            filtered_tokens.append(token)
    if stopwords:
        stems = [stemmer.stem(t) for t in filtered_tokens if t not in stopwords]
    else:
        stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text, stopwords=None):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [
        word.lower()
        for sent in nltk.sent_tokenize(text)
        for word in nltk.word_tokenize(sent)
    ]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search("[a-zA-Z]", token):
            filtered_tokens.append(token)
    if stopwords:
        r = [i for i in filtered_tokens if i not in stopwords]
    else:
        r = filtered_tokens
    return r

def top_n_sim(award_desc, award_desc_dict, sim_mat, n):
    idx = int([i for i, v in award_desc_dict.items() if v == award_desc][0])
    v = sim_mat[idx, :]
    n_idx = np.argpartition(v, -n)[-n:]
    r = [award_desc_dict[str(i)] for i in np.array(n_idx)]
    return r

def bert_sent_embedding(sentence_list, hidden_layer=-2):
    # Load pre-trained model tokenizer (vocabulary)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    # Tokenize and return tensors
    batch = tokenizer(sentence_list, padding=True, truncation=True, return_tensors="pt")
    # Load pre-trained model (weights)
    model = BertModel.from_pretrained(
        "bert-base-uncased",
        output_hidden_states=True,
    )
    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()
    # Run the text through BERT, and collect all of the hidden states produced
    # from all layers.
    with torch.no_grad():
        outputs = model(**batch)
        # Evaluating the model will return a different number of objects based on
        # how it's  configured in the `from_pretrained` call earlier. In this case,
        # becase we set `output_hidden_states = True`, the third item will be the
        # hidden states from all layers.
        hidden_states = outputs[2]
    # Extract the second to last layer
    token_vecs = hidden_states[hidden_layer]
    # Calculate the average of all sentence token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding


def pairwise_cosine_similarity(tensor, feature_dim=1, use_cuda=None):
    if isinstance(tensor, np.ndarray):
        print("Using Numpy")
        n = np.linalg.norm(tensor, axis=feature_dim, keepdims=True)
        norm_tensor = tensor / n
        r = np.dot(norm_tensor, norm_tensor.T)
    elif isinstance(tensor, torch.Tensor):
        if use_cuda:
            print("Using PyTorch with CUDA")
            tensor = tensor.cuda()
        else:
            print("Using PyTorch with CPU")
        n = torch.norm(tensor, dim=feature_dim, keepdim=True)
        norm_tensor = tensor / n
        r = torch.matmul(norm_tensor, norm_tensor.T)
    else:
        raise ("Unknown input data type")
    return r

def top_n_sim(award_desc, award_desc_dict, sim_mat, n):
    idx = int([i for i, v in award_desc_dict.items() if v == award_desc][0])
    v = sim_mat[idx, :]
    n_idx = np.argpartition(v, -n)[-n:]
    r = [award_desc_dict[str(i)] for i in np.array(n_idx)]
    return r

def similar_records_bert(
    top_n, award_description, records=None, model="distilbert-base-nli-mean-tokens"
):
    if model in (
        "distilbert-base-nli-mean-tokens",
        "paraphrase-distilroberta-base-v1",
        "paraphrase-xlm-r-multilingual-v1",
        "stsb-roberta-large",
        "stsb-roberta-base",
        "stsb-distilbert-base",
        "stsb-bert-large",
        "msmarco-distilroberta-base-v2",
        "msmarco-roberta-base-v2",
        "msmarco-distilbert-base-v2",
        "nq-distilbert-base-v1",
    ):
        model = SentenceTransformer(model)
        record_embedding = model.encode(records["award_description"])
        new_case_embedding = model.encode(award_description)
        sim = cosine_similarity(new_case_embedding, record_embedding)
    else:
        new_case_embedding = bert_sent_embedding(award_description)
        sim = cosine_similarity(
            new_case_embedding,
            records[records.columns[records.columns.str.contains("bert_feat_")]],
        )
    r = []
    if isinstance(award_description, list):
        for i in range(sim.shape[0]):
            n_idx = np.argpartition(sim[i, :], -top_n)[-top_n:]
            r.append(records.iloc[n_idx])
    else:
        n_idx = np.argpartition(sim[0, :], -top_n)[-top_n:]
        r = records.iloc[n_idx]
    return r

## Process award description of IRS contracts 

In [3]:
# read in all contracts
dfs = []
for fp in glob.glob('../data/treasury/*.zip'):
    if '2019' in fp or '2020' in fp:
        df = pd.read_csv(fp, low_memory=False, dtype={'modification_number':str, 'parent_award_agency_id':str, 'awarding_sub_agency_code':str, 'funding_sub_agency_code':str})
        df['fy'] = os.path.basename(fp)[2:6]
        dfs.append(df)
dat = pd.concat(dfs)

# IRS contracts only
dat = dat.loc[(dat.modification_number=='0')&((dat.parent_award_agency_id=='2050')|(dat.awarding_sub_agency_code=='2050')|(dat.funding_sub_agency_code=='2050'))]
# treasury contracts
# dat = dat.loc[(dat.modification_number=='0')]

dat['action_date'] = pd.to_datetime(dat['action_date'])
dat['solicitation_date'] = pd.to_datetime(dat['solicitation_date'])
dat['palt'] = (dat['action_date'] - dat['solicitation_date']).dt.days
dat = dat.loc[dat.palt.notnull()].reset_index(drop=True)

# # select 95%tile for outlier cutoff
palt = dat.loc[dat.palt<500].reset_index(drop=True)
print(dat.shape)

(924, 284)


## Extract contextualized word embedding features using BERT

In [4]:
# write award_description into json file
batch_sent = dat.award_description.tolist()

with open('bert/award_desc.json', 'w') as f:
    json.dump({i:v for i, v in enumerate(batch_sent)}, f)
    
# create award_description dictionary
with open('bert/award_desc.json', 'r') as f:
    award_desc_dict = json.load(f)
    
batches = divide_chunks(batch_sent, 100)

# bert embedding
%time bert_embed = [bert_sent_embedding(i) for i in tqdm(batches)]

bert_embeddings = torch.cat(bert_embed, 0)

bert_feat = pd.DataFrame(bert_embeddings.numpy(), columns=['bert_feat_'+str(i) for i in range(bert_embeddings.shape[1])])
bert_feat.to_csv('bert/bert_embed.csv', index=False)

10it [01:49, 11.00s/it]


Wall time: 1min 49s


## Measure and rank order similarity between IRS contracts and any individual contract

In [6]:
# read in embeading features
# bert_feat = pd.read_csv('bert/bert_embed.csv')

# calculate cosine similarity of all contracts
cosine_sim = pairwise_cosine_similarity(bert_feat.values)

# top n contracts w. high similiarity score for contract of interest
top_n_sim('SPLUNK SUPPORT WITH OPTION YEARS', award_desc_dict, cosine_sim, 20)

Using Numpy


['WMQ BILLABLE SOFTWARE RENEWAL',
 'NSHIELD CONNECT 6000+F3(PREMIUM SUPPORT)',
 'ADVANCE EDITION SERVER LICENSE MAINTENANCE PACK',
 'RRAC ACCESS-GEN LICENSE RENEWAL',
 'AUTHENTIC8 SECURE INTERNET BROWSER ACCESS SOFTWARE AND SUPPORT-BASE PLUS TWO (2) OPTIONS',
 'IDG SERVERS SUPPORT',
 'ADMINSTUDIO SOFTWARE MAINTENANCE RENEWAL (12 MONTHS)',
 'ACA DATA ANALYSIS - BASE YEAR + 4 OPTION YEARS',
 'SAN BROCADE SWITCH SUPPORT',
 'TRUE UP- MID SERVICE UPGRADE FROM STANDARD TO PREMIUM SUPPORT',
 'VMWARE SUPPORT',
 'I2 ANALYSTS NOTEBOOK SW SUBSCRIPTION AND SUPPORT FY20 RENEWAL',
 'SPLUNK SUPPORT WITH OPTION YEARS',
 'DELOITTE - ECM WEBAPPS REQUIREMENT 6MONTH BASE W/6 MONTH OPTION',
 'THIS TASK ORDER AWARD IS FOR EST WEBAPP SUPPORT SERVICE 6 MONTH BASE (FUNDED) WITH 6 MONTH OPTION (UNFUNDED)',
 'WEB SUBSCRIPTION 1 YEAR SERVICE',
 'FFP FIXED PRICE FOR PLOTTER AND INSTALLER WITH ENCRYPTED HD',
 "IBM I2 ANALYST'S NOTEBOOK CONCURRENT USER LICENSE + SW SUBSCRIPTION&SUPPORT 12 MONTHS",
 'WEBMETHODS EAIB 

## Extract word embeddings of Splunk contracts

In [7]:
splunk = pd.read_csv('data/USA3_Splunk_slim.csv')
splunk_feat = bert_sent_embedding(splunk.award_description.tolist())

## Measure and rank order similarity between IRS contracts and Splunk contracts

In [8]:
# calculate cosine similarity for each of splunk case with each of the data cases and then take mean
dat['BERT cosine similarity'] = cosine_similarity(splunk_feat, bert_feat).mean(axis=0)

# rank order
dat = dat.sort_values(by="BERT cosine similarity", ascending=False)

dat[["award_id_piid","award_description", "BERT cosine similarity"]]

,award_id_piid,award_description,BERT cosine similarity
124,2032H519F00784,WMQ BILLABLE SOFTWARE RENEWAL,0.926749
622,2032H520F00499,ZEVA DECRYPTNABOX MAINTENANCE,0.922714
639,2032H520F00591,SILO SOFTWARE RENEWAL FOR CRIMINAL INVESTIGATION,0.921596
151,2032H519F00790,SILO SOFTWARE LICENSE AND SUPPORT FOR CRIMINAL...,0.920372
732,2032H520F00930,EBS GSOC SECOPS SOLUTION,0.919565
...,...,...,...
360,2032H520F00031,AUDIOCODES MEDIA GATEWAYS FOR EEFAX BASE PERIO...,0.692216
795,2032H520F00906,THE INTERNAL REVENUE SERVICE (IRS) INFORMATION...,0.691590
619,2032H520F00496,ENTERPRISE CASE MANAGEMENT (ECM) BPA TASK ORDE...,0.688282
255,2032H819F00229,THE PURPOSE OF THIS DELIVERY ORDER IS EXECUTE&...,0.667566


In [9]:
# output
dat[["award_id_piid","award_description", "BERT cosine similarity"]].to_csv('output/splunk_similarity_bert.csv', index=False)

## Measure and rank order similarity between IRS contracts and any award description

In [ ]:
# manually input award description 
award_description_rand = ['SPLUNK LICENCE RENEWAL']
records = dat[["award_id_piid","award_description"]]

r = similar_records_bert(20, award_description_rand, records, model='paraphrase-distilroberta-base-v1')

for i, v in enumerate(award_description_rand):
    print(r[i][['award_id_piid','award_description']], end =" ")